#### Notebook for running Chain-of-Thought with no supporting context experiments

In [1]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-proj-o5678901234567890'

In [2]:
import sys, os
sys.path.append('..')
root = '../root/'

In [3]:
from util import summarize_trial, log_trial, save_agents
import joblib
from agents import CoTAgent, ReflexionStrategy

() {'temperature': 0, 'max_tokens': 250, 'stop': ['\n']} {'temperature': 0, 'max_tokens': 250, 'stop': ['\n']}
() {'temperature': 0, 'max_tokens': 250, 'stop': ['\n']} {'temperature': 0, 'max_tokens': 250, 'stop': ['\n']}
() {'temperature': 0, 'max_tokens': 100, 'stop': ['\n']} {'temperature': 0, 'max_tokens': 100, 'stop': ['\n']}
() {'temperature': 0, 'max_tokens': 100, 'stop': ['\n']} {'temperature': 0, 'max_tokens': 100, 'stop': ['\n']}
() {'temperature': 0, 'max_tokens': 250} {'temperature': 0, 'max_tokens': 250}


#### Load the HotPotQA Sample

In [4]:
hotpot = joblib.load('../data/hotpot-qa-distractor-sample.joblib').reset_index(drop = True)

#### Define the Reflexion Strategy

In [5]:
print(ReflexionStrategy.__doc__)


    NONE: No reflection
    LAST_ATTEMPT: Use last reasoning trace in context 
    REFLEXION: Apply reflexion to the next reasoning trace 
    LAST_ATTEMPT_AND_REFLEXION: Use last reasoning trace in context and apply reflexion to the next reasoning trace 
    


In [6]:
strategy: ReflexionStrategy = ReflexionStrategy.REFLEXION

#### Initialize a CoTAgent for each question

In [7]:
from prompts import cot_simple_reflect_agent_prompt, cot_simple_reflect_prompt, cot_simple_agent_prompt
from fewshots import COTQA_SIMPLE6, COT_SIMPLE_REFLECTION

agents = [CoTAgent(question = row['question'],
                   context = '',
                   key = row['answer'],
                   agent_prompt=cot_simple_agent_prompt if strategy == ReflexionStrategy.NONE else cot_simple_reflect_agent_prompt,
                   cot_examples = COTQA_SIMPLE6,
                   reflect_prompt = cot_simple_reflect_prompt,
                   reflect_examples = COT_SIMPLE_REFLECTION,
                      ) for _, row in hotpot.iterrows()]

#### Run `n` trials

In [8]:
n = 5
trial = 0
log = ''
for i in range(n):
    for agent in [a for a in agents if not a.is_correct()]:
        agent.run(reflexion_strategy = strategy)
        print(f'Answer: {agent.key}')
    trial += 1
    log += log_trial(agents, trial)
    correct, incorrect = summarize_trial(agents)
    print(f'Finished Trial {trial}, Correct: {len(correct)}, Incorrect: {len(incorrect)}')

> prompt(chat): Solve a question answering task by having a Thought, then Finish with your answer. Thought can reason about the current situation. Finish[answer] returns the answer and finishes the task.
Here are some examples:

Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: Let's think step by step. The eastern sector of Colorado orogeny extends into the High Plains. High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]

Question: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
Thought: Let's think step by step. Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.
Action: Finish[Richard Nixon]

Question: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
Thought: Let's think step by s

TypeError: cannot unpack non-iterable NoneType object

#### Save the result log

In [27]:
with open(os.path.join(root, 'CoT', 'no_context', strategy.value, f'{len(agents)}_questions_{trial}_trials.txt'), 'w') as f:
    f.write(log)
save_agents(agents, os.path.join(root, 'CoT', 'no_context', strategy.value, 'agents'))